In [3]:
import os
import cv2
import copy
import numpy as np

In [4]:
# 椭圆拟合及筛选
def ellipse_fit_filter(image, contours):
    """
    Args:
        image (_type_): init image
        contours (_type_): canny edge detection
    """
    
    # save circle and bounding box
    result = []
    
    # detect circle
    processed_centers = []
    
    for contour in contours:
        # 如果轮廓点数不足 5，则跳过
        if len(contour) < 5:
            continue

        # 根据圆的面积筛选
        area = cv2.contourArea(contour)
        if area < 5 or area > 200:
                continue
        
        # 根据圆形度筛选   
        perimeter = cv2.arcLength(contour, True)
        circularity = 4 * np.pi * (area / (perimeter * perimeter))
        if circularity < 0.8:
            continue
        
        # 圆形度应接近 1，设置合适的阈值
        if circularity < 0.7:
            continue
        # 椭圆拟合
        ellipse = cv2.fitEllipse(contour)

        # 计算轮廓的几何矩
        M = cv2.moments(contour)
        if M['m00'] != 0:
            cx = M['m10'] / M['m00']
            cy = M['m01'] / M['m00']
        else:
            continue
        
        # 合并相近的圆心
        close_enough = False
        for center in processed_centers:
            if np.linalg.norm(np.array(center) - np.array((cx, cy))) < 10:  # 设定距离阈值
                close_enough = True
                break

        if close_enough:
            continue  # 如果相近则跳过

        # 拟合椭圆
        ellipse = cv2.fitEllipse(contour)
        w, h = ellipse[1][0], ellipse[1][1]

        # 计算 bounding box 的左上角坐标
        top_left_x = int(cx - w / 2)
        top_left_y = int(cy - h / 2)

        # 在图像上绘制 bounding box
        cv2.rectangle(image, (top_left_x, top_left_y), (top_left_x + int(w), top_left_y + int(h)), (255, 0, 0), 2)

        # 标注圆心
        int_center = (int(cx), int(cy))
        center = (cx, cy)
        cv2.circle(image, int_center, 3, (0, 0, 255), -1)
        
        # 记录处理后的圆心
        processed_centers.append((cx, cy))
        result.append((cx, cy, w, h))
        # 输出bounding box的宽高和圆心坐标
        # print(f"Center: {center}, Bounding Box (w, h): ({w}, {h})")

    # # 显示结果
    # cv2.imshow('Fitted Ellipses', image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    
    return result

In [5]:
# Calculate normalized coordinates and store in label.txt
def coordinate_norm_and_save(bounding_boxes, image_width, image_height, save_path, image_filename):
    # 存储归一化的 bounding box
    normalized_bounding_boxes = []

    for bbox in bounding_boxes:
        # 计算归一化的中心坐标
        x_center_normalized = bbox[0] / image_width
        y_center_normalized = bbox[1] / image_height
        
        # 计算归一化的宽度和高度
        w_normalized = bbox[2] / image_width
        h_normalized = bbox[3] / image_height
        
        # 将归一化结果存储在列表中
        normalized_bounding_boxes.append({
            'class_id': 0,
            'x_center': x_center_normalized,
            'y_center': y_center_normalized,
            'width': w_normalized,
            'height': h_normalized
        })
        
    # 创建标签文件路径
    label_filename = os.path.join(save_path, f"{image_filename}.txt")

    # 写入归一化结果到文本文件
    with open(label_filename, 'w') as f:
        for bbox in normalized_bounding_boxes:
            # 按要求格式写入每一行
            line = f"{bbox['class_id']} {bbox['x_center']} {bbox['y_center']} {bbox['width']} {bbox['height']}\n"
            f.write(line)
            

In [6]:
def circle_detect(save_path, image_filename, image):
    
    iter = 3
    
    threshold_difference = 20
    
    for i in range(iter):     
        # 转为灰度图像
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # 高斯模糊去噪
        blurred = cv2.GaussianBlur(gray, (5, 5), 1.5)

        # 使用 Canny 进行边缘检测
        edges = cv2.Canny(blurred, threshold_difference, 255)

        # 查找轮廓
        contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        center_bounding_box = ellipse_fit_filter(image, contours)
        
        coordinate_norm_and_save(center_bounding_box, image.shape[1], image.shape[0], save_path, image_filename)


In [7]:
# 回调函数，用于鼠标事件
def select_roi(event, x, y, flags, param):
    global roi_selected, roi_start, roi_end, selecting

    if event == cv2.EVENT_LBUTTONDOWN:
        roi_start = (x, y)
        selecting = True
    elif event == cv2.EVENT_MOUSEMOVE and selecting:
        roi_end = (x, y)
    elif event == cv2.EVENT_LBUTTONUP:
        roi_end = (x, y)
        roi_selected = True
        selecting = False

In [1]:
video_path = 'video_data\TC_S1_acting1_cam1.mp4'
imgs_save_path = 'data\images'
labels_save_path = 'data\labels'

video_name = video_path.split('\\')[1].split('.')[0]
imgs_save_path + '\\' + video_name + '_' + str(1) + '.bmp'

'data\\images\\TC_S1_acting1_cam1_1\\.bmp'

In [9]:
# 初始化
roi_selected = False
selecting = False
roi_start = (0, 0)
roi_end = (0, 0)

# 打开视频文件
cap = cv2.VideoCapture(video_path)

cv2.namedWindow("Frame")
# cv2.setMouseCallback("Frame", select_roi)

index = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    paint_img = copy.deepcopy(frame)
    
    circle_center = circle_detect(labels_save_path, video_name + str(index), frame)

    # 检测键盘输入
    key = cv2.waitKey(0) & 0xFF
    if key == ord('c'):  # 按下 'c' 表示接受当前帧
        print("圆心检测通过，处理当前帧...")
        # 可以在此保存处理结果或进行其他操作
        cv2.imwrite(imgs_save_path + '/' + video_name + '_' + str(index) + '/.bmp', frame)
        index += 1

    elif key == ord('d'):  # 按下 'd' 表示跳过当前帧
        print("跳过当前帧")
        
    elif key == ord('q'):  # 按下 'q' 退出
        break
    
    # # 如果ROI已经选择
    # if roi_selected:
    #     x1, y1 = roi_start
    #     x2, y2 = roi_end

    #     # 画出选中的ROI矩形
    #     cv2.rectangle(paint_img, roi_start, roi_end, (0, 255, 0), 2)

    #     # 提取ROI区域
    #     roi = paint_img[y1:y2, x1:x2]

    #     # 检测圆心
    #     circle_center = circle_detect(labels_save_path, video_name + str(index), frame, roi)
    #     if circle_center is not None:
    #         cx, cy = circle_center
    #         # 将圆心坐标映射回全局帧坐标系
    #         global_center = (x1 + cx, y1 + cy)
    #         # 在原始帧上标记圆心
    #         cv2.circle(paint_img, global_center, 5, (0, 0, 255), -1)
        
    #     # 显示处理后的帧
    #     cv2.imshow("Frame", paint_img)

    #     # 检测键盘输入
    #     key = cv2.waitKey(0) & 0xFF
    #     if key == ord('c'):  # 按下 'c' 表示接受当前帧
    #         print("圆心检测通过，处理当前帧...")
    #         # 可以在此保存处理结果或进行其他操作
    #         cv2.imwrite(imgs_save_path + video_name + str(index), frame)
    #         index += 1

    #     elif key == ord('d'):  # 按下 'd' 表示跳过当前帧
    #         print("跳过当前帧")
    #         roi_selected = False  # 重新选择ROI
    #     elif key == ord('q'):  # 按下 'q' 退出
    #         break
    # else:
    #     # 如果尚未选择ROI，显示当前帧
    #     cv2.imshow("Frame", paint_img)

    #     # 检测键盘输入
    #     key = cv2.waitKey(0) & 0xFF
    #     if key == ord('q'):  # 按下 'q' 退出
    #         break

cap.release()
cv2.destroyAllWindows()

NameError: name 'roi' is not defined

: 

In [9]:
# folder_path = 'data/ball'
# save_path = 'label'

# for filename in os.listdir(folder_path):
#     # 检查文件扩展名
#     if filename.endswith('.png'):
#         # 构建文件的完整路径
#         file_path = os.path.join(folder_path, filename)
        
#         # 使用 OpenCV 加载图像
#         image = cv2.imread(file_path)
        
#         # 获取图片无后缀名
#         filename_without_bmp = os.path.splitext(filename)[0]

#         # 检查图像是否成功加载
#         if image is not None:
#             circle_detect(save_path=save_path, image_filename=filename_without_bmp, image=image, circle_nums=1)
#             print(f"Loaded: {filename}")
#         else:
#             print(f"Failed to load: {filename}")
